In [ ]:
import os
import joblib
import pandas as pd
import datetime

import seaborn as sns

In [ ]:
sns.set(rc={'figure.figsize':(24,8.27)})

In [ ]:
DATA_OUTPUT_DIR = os.path.join('..','00.data','output')


In [ ]:
if "DATA_OUTPUT_DIR" in os.environ:
    DATA_OUTPUT_DIR = os.environ["DATA_OUTPUT_DIR"]


In [ ]:
files_found = [x for x in os.listdir(DATA_OUTPUT_DIR) if x.endswith('.pickle')]
files_found_tokens=[x.split('.') for x in files_found]

models_found = {}

for current_model in files_found_tokens:
    model_name = '.'.join(current_model[:-2])
    filename = os.path.join(DATA_OUTPUT_DIR,'.'.join(current_model))
    if not model_name in models_found:
        models_found[model_name]= {}
    models_found[model_name][current_model[-2]]=filename


In [ ]:
first_model = list(models_found.keys())[0]
current_parameters = joblib.load(models_found[first_model]['parameters'])
MINIMUM_TIME = current_parameters['MINIMUM_TIME']
MAXIMUM_TIME = current_parameters['MAXIMUM_TIME']
CURRENT_EXCHANGE = current_parameters['CURRENT_EXCHANGE']
CURRENT_ASSET = current_parameters['CURRENT_ASSET']
CURRENT_TIMEFRAME= current_parameters['CURRENT_TIMEFRAME']
CURRENT_TARGET= current_parameters['CURRENT_TARGET']
CURRENT_STOP= current_parameters['CURRENT_STOP']
MAX_TRADE_DURATION = current_parameters['MAX_TRADE_DURATION']
DECISION_BOUNDARY = current_parameters['DECISION_BOUNDARY']


In [ ]:
raw_trades_file_name = os.path.join(f"{CURRENT_EXCHANGE}.{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.test_trades.xlsx")
raw_trades_full_file_name = os.path.join(DATA_OUTPUT_DIR,raw_trades_file_name)
trades = pd.read_excel(raw_trades_full_file_name).to_dict('records')

trades


## Overall Trade Results

In [ ]:
index_col = list(trades[0].keys())[0]
trades_result = [{'index' : x[index_col]  ,'result' : x['result']} for x in trades]

current_trade_total = 0
trades_total = []
for current_trade in trades_result:
    trades_total.append({ 'index': current_trade['index'], 'total': current_trade_total })
    current_trade_total += int(current_trade['result'])

sns.lineplot(data=pd.DataFrame(trades_total), x="index", y="total")


## Trade Results per Month

In [ ]:

current_trades_month = {}

for current_trade in trades:
    current_month=str(int(current_trade['Date'] / 100))
    if current_month in current_trades_month:
        current_trades_month[current_month]+=int(current_trade['result'])
    else:
        current_trades_month[current_month]=int(current_trade['result'])

result_per_month = []

for current_month in list(current_trades_month.keys()):
    result_per_month.append({ 'month' : current_month, 'result': current_trades_month[current_month]})

sns.barplot(pd.DataFrame(result_per_month), x="month", y="result")


## Trade Results, but with Max Loss Stop

In [ ]:

loss_scenarios = [-10, -20,-30,-40,-50,-60,-70,-80,-90,-100, -200, -400, -800]
current_result_max_stop = {}

for max_loss in loss_scenarios:
    date_results = {}
    for current_trade in trades:
        current_date = current_trade['Date']
        if current_date in date_results:
            if date_results[current_date] < max_loss:
                continue
            else:
                date_results[current_date]+=int(current_trade['result'])
        else:
            date_results[current_date]=int(current_trade['result'])

    total = 0
    for current_date in list(date_results.keys()):
        total+=date_results[current_date]

    current_result_max_stop[str(max_loss)]=total

result_per_max_loss = []
for current_max_loss in list(current_result_max_stop.keys()):
    result_per_max_loss.append({ 'max_loss' : current_max_loss, 'result': current_result_max_stop[current_max_loss] - current_trade_total})

ax = sns.barplot(pd.DataFrame(result_per_max_loss), x="max_loss", y="result")

for i in ax.containers:
    ax.bar_label(i, )


## Trade Results per Hour

In [ ]:
current_trades_hour = {}

for current_trade in trades:
    current_hour=str(int(current_trade['Time'] )).zfill(4)
    if current_hour in current_trades_hour:
        current_trades_hour[current_hour]+=int(current_trade['result'])
    else:
        current_trades_hour[current_hour]=int(current_trade['result'])

result_per_month = []

keys_sorted = list(current_trades_hour.keys())

keys_sorted.sort()

for current_hour in keys_sorted:
    result_per_month.append({ 'hour' : current_hour, 'result': current_trades_hour[current_hour]})

sns.barplot(pd.DataFrame(result_per_month), x="hour", y="result")


## Trade Results per WeekDay

In [ ]:
current_trades_weekday = {}

for current_trade in trades:

    current_date_str = str(current_trade['Date'])
    current_date_year = int(current_date_str[0:4])
    current_date_month = int(current_date_str[4:6])
    current_date_day = int(current_date_str[6:8])

    current_weekday=datetime.date(current_date_year, current_date_month, current_date_day).weekday()

    if current_weekday in current_trades_weekday:
        current_trades_weekday[current_weekday]+=int(current_trade['result'])
    else:
        current_trades_weekday[current_weekday]=int(current_trade['result'])

result_per_weekday = []

keys_sorted = list(current_trades_weekday.keys())

keys_sorted.sort()

for current_weekday in keys_sorted:
    result_per_weekday.append({ 'weekday' : current_weekday, 'result': current_trades_weekday[current_weekday]})

sns.barplot(pd.DataFrame(result_per_weekday), x="weekday", y="result")
